In [3]:
import requests
import json
import pandas as pd
import re

ollama_url_gen = "http://localhost:11434/api/generate"
ollama_model_name = "phi3"

def generate_toc_prompt(subject):
    return f"""
You are an expert assistant specializing in creating structured and logical book outlines.
Your task is to generate a detailed Table of Contents for a book on the given subject.
Each chapter must have a title, and chapters can include subsections and examples where applicable.
The Table of Contents should be well-structured, logically ordered, and strictly formatted as a valid JSON object without any extra text.

Format:
{{
    "Table_of_Contents": [
        {{
            "Chapter 1": "Title of Chapter",
            "Sections": [
                "Section 1.1: Title of Section",
                "Section 1.2: Title of Section"
            ]
        }},
        {{
            "Chapter 2": "Title of Chapter",
            "Sections": [
                "Section 2.1: Title of Section",
                "Section 2.2: Title of Section"
            ]
        }}
    ]
}}

Subject: {subject}
"""
    payload = {"model": model, "prompt": prompt, "stream": False}
    headers = {"Content-Type": "application/json"}
    response = requests.post(ollama_url_gen, headers=headers, data=json.dumps(payload))
    
    if response.status_code != 200:
        raise Exception(f"Error from Ollama: {response.text}")
    
    content = response.json().get("response", "")
    try:
        # Extract potential JSON array or object
        json_match = re.search(r"\{.*\}|\[.*\]", content, re.DOTALL)
        if json_match:
            raw_json = json_match.group()
            
            # Attempt to fix common issues, such as missing commas between JSON objects
            sanitized_json = re.sub(r'(\}|"[^"]*")\s*("[^"]*":)', r'\1,\2', raw_json)
            return json.loads(sanitized_json)
        else:
            raise Exception(f"Valid JSON not found in response: {content}")
    except json.JSONDecodeError as e:
        raise Exception(f"Failed to parse JSON response after sanitization: {sanitized_json}") from e

try:
    toc = generate_toc("Prompt Engineering")
    print(json.dumps(toc, indent=4))
except Exception as e:
    print(str(e))


In [4]:
generate_toc("Prompt Engineering") Please return the JSON strictly formatted, with commas between all sections, and avoid including any additional text outside the JSON object.


Exception: Failed to parse JSON response: {
    "Table_of_Contents": [
        {
            "Chapter 1": "Introduction to Prompt Engineering"
        },
        {
            "Chapter 2": "History and Evolution of Chatbots",
            "Section 2.1": "Early Developments in AI Communication",
            "Section 2.2": "The Advent of Natural Language Processing (NLP)"
        },
        {
            "Chapter 3": "Principles and Techniques of Prompt Engineering"
            "Section 3.1": "Understanding the Principles behind Chatbot Interaction",
            "Section 3.2": "Various Strategies in Crafting Effective Prompts",
            "Example Section 3.2.a": "Case Studies on Successful and Failed Communication with AI"        
        },
        {
            "Chapter 4": "Data Collection and Analysis for Optimizing Chatbot Responses"
            "Section 4.1": "Methods of Data Gathering",
            "Section 4.2": "Analyze Patterns in AI Conversations to Improve Interaction Qualit